In [26]:
from dotenv import load_dotenv
load_dotenv() # .env 파일에서 환경변수 로드

True

In [27]:
import os
from glob import glob 

from pprint import pprint 
import json 

## 1. 벡터 저장소 (Vector Store)
- 텍스트 데이터를 숫자 벡터로 변환(임베딩)하여 저장하고, 유사도 기반으로 빠르게 검색할 수 있게 해주는 시스템임. 
- RAG(Retrieval Augmented Generation)의 핵심 구성 요소 중 하나임.

### 1.1 Chroma

- 사용자 편의성이 우수한 오픈소스 벡터 저장소임. 
- 개발 및 테스트 환경에서 빠르게 프로토타이핑하기 좋음.

- **장점:**
  - 사용 편의성: API가 직관적이고 LangChain과 통합이 잘 되어 있음.
  - 인메모리 및 영구 저장 지원: 개발 시 빠르게 테스트하고, 필요시 디스크에 저장하여 데이터 유지 가능.
  - 다양한 검색 옵션 제공: 유사도 검색, MMR, 메타데이터 필터링 등.
- **단점:**
  - 대규모 데이터셋 처리: 수백만 건 이상의 매우 큰 데이터셋에서는 FAISS 같은 최적화된 라이브러리보다 성능이 떨어질 수 있음.
  - 분산 환경 지원: 기본적으로 단일 노드에서 실행되도록 설계됨. (ChromaDB Cloud 같은 서비스는 분산 지원)

`(1) 벡터 저장소 초기화`

- 문서를 벡터로 변환할 임베딩 모델을 선택하고, Chroma DB를 초기화함.
- `BAAI/bge-m3` 모델은 다국어 지원 및 성능이 우수한 임베딩 모델 중 하나임.
- `persist_directory`를 지정하면 DB 변경 사항이 해당 디렉토리에 저장되어 나중에 다시 로드할 수 있음.

In [28]:
# 벡터 저장소에 문서를 저장할 때 적용할 임베딩 모델
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
# 강력한 다국어 임베딩 모델
embeddings_model = HuggingFaceEmbeddings(model_name="BAAI/bge-m3") 

# 벡터 저장소 생성
from langchain_chroma import Chroma

chroma_db = Chroma(
    collection_name="ai_smaple_collection", # DB 내 컬렉션 이름
    embedding_function=embeddings_model,    # 사용할 임베딩 함수
    persist_directory="./chroma_db",        # 저장 경로 (영속화)
)

print("Chroma DB 초기화 완료. 저장 경로: ./chroma_db")

Chroma DB 초기화 완료. 저장 경로: ./chroma_db


In [29]:
# 현재 컬렉션 정보 확인 (초기에는 비어있을 수 있음)
pprint(chroma_db.get())

{'data': None,
 'documents': ['인공지능(AI)은 컴퓨터 과학의 핵심 분야로, 기계학습과 딥러닝을 포괄합니다.',
               '머신러닝(ML)은 데이터로부터 학습하여 예측과 결정을 내리는 인공지능의 중요한 하위 분야입니다.',
               '딥러닝(DL)은 머신러닝의 한 갈래로, 복잡한 패턴 인식을 위해 심층 신경망을 사용합니다.',
               '자연어 처리는 컴퓨터가 인간의 언어를 이해하고 생성하는 기술입니다.'],
 'embeddings': None,
 'ids': ['DOC_1', 'DOC_2', 'DOC_3', 'DOC_4'],
 'included': [<IncludeEnum.documents: 'documents'>,
              <IncludeEnum.metadatas: 'metadatas'>],
 'metadatas': [{'chapter': 'Chapter 1', 'source': 'AI_textbook_v2'},
               {'chapter': 'Chapter 2', 'source': 'AI_textbook_v2'},
               {'chapter': 'Chapter 3', 'source': 'AI_textbook_v2'},
               {'chapter': 'Chapter 4', 'source': 'AI_textbook'}],
 'uris': None}


`(2) 벡터 저장소 관리`

Chroma DB에 문서를 추가, 수정, 삭제하는 방법을 알아봄.

- 문서 추가: `vector_store.add_documents(documents, ids)`
  - `Document` 객체 리스트와 각 문서에 대한 고유 ID 리스트를 전달하여 저장함.
  - 메타데이터를 통해 문서의 출처, 챕터 등 부가 정보를 저장할 수 있음.
  - **장점:** 일괄 추가 가능, 메타데이터 활용 용이.
  - **단점:** ID는 고유해야 하며, 중복 시 오류 발생 가능 (Chroma 설정에 따라 다를 수 있음).

In [30]:
from langchain_core.documents import Document

# 샘플 문서 내용
documents_content = [
    "인공지능은 컴퓨터 과학의 한 분야입니다.",
    "머신러닝은 인공지능의 하위 분야입니다.",
    "딥러닝은 머신러닝의 한 종류입니다.",
    "자연어 처리는 컴퓨터가 인간의 언어를 이해하고 생성하는 기술입니다.",
    "컴퓨터 비전은 컴퓨터가 디지털 이미지나 비디오를 이해하는 방법을 연구합니다."
]

# Document 객체로 변환
doc_objects = []
for i, content in enumerate(documents_content, start=1):
    doc = Document(
        page_content=content,
        metadata={"source": "AI_textbook", "chapter": f"Chapter {i}"}, # 메타데이터 추가
    )
    doc_objects.append(doc)

# 각 문서에 대한 고유 ID 생성
doc_ids = [f"DOC_{i}" for i in range(1, len(doc_objects) + 1)]

# 문서를 벡터 저장소에 추가
added_doc_ids = chroma_db.add_documents(documents=doc_objects, ids=doc_ids)

print(f"{len(added_doc_ids)}개의 문서가 성공적으로 벡터 저장소에 추가되었습니다.")
print("추가된 문서 ID:", added_doc_ids)

5개의 문서가 성공적으로 벡터 저장소에 추가되었습니다.
추가된 문서 ID: ['DOC_1', 'DOC_2', 'DOC_3', 'DOC_4', 'DOC_5']


In [31]:
# 추가 후 문서 검색 테스트
query = "인공지능과 머신러닝의 관계는?"
results = chroma_db.similarity_search(query, k=2) # 가장 유사한 2개 문서 검색

print(f"\n쿼리: {query}")
print("검색된 유사 문서:")
for doc in results:
    print(f"- {doc.page_content} [출처: {doc.metadata['source']}, {doc.metadata['chapter']}]")


쿼리: 인공지능과 머신러닝의 관계는?
검색된 유사 문서:
- 머신러닝은 인공지능의 하위 분야입니다. [출처: AI_textbook, Chapter 2]
- 딥러닝은 머신러닝의 한 종류입니다. [출처: AI_textbook, Chapter 3]


In [32]:
# 현재 저장된 컬렉션 데이터 확인 (문서 추가 후)
pprint(chroma_db.get(include=['metadatas', 'documents'])) # ids, embeddings 등도 포함 가능

{'data': None,
 'documents': ['인공지능은 컴퓨터 과학의 한 분야입니다.',
               '머신러닝은 인공지능의 하위 분야입니다.',
               '딥러닝은 머신러닝의 한 종류입니다.',
               '자연어 처리는 컴퓨터가 인간의 언어를 이해하고 생성하는 기술입니다.',
               '컴퓨터 비전은 컴퓨터가 디지털 이미지나 비디오를 이해하는 방법을 연구합니다.'],
 'embeddings': None,
 'ids': ['DOC_1', 'DOC_2', 'DOC_3', 'DOC_4', 'DOC_5'],
 'included': [<IncludeEnum.documents: 'documents'>,
              <IncludeEnum.metadatas: 'metadatas'>],
 'metadatas': [{'chapter': 'Chapter 1', 'source': 'AI_textbook'},
               {'chapter': 'Chapter 2', 'source': 'AI_textbook'},
               {'chapter': 'Chapter 3', 'source': 'AI_textbook'},
               {'chapter': 'Chapter 4', 'source': 'AI_textbook'},
               {'chapter': 'Chapter 5', 'source': 'AI_textbook'}],
 'uris': None}


- 문서 수정: `vector_store.update_document(document_id, document)` 또는 `vector_store.update_documents(ids, documents)`
  - 특정 ID의 문서를 새로운 `Document` 객체로 교체함.
  - **장점:** 개별 또는 여러 문서 업데이트 용이.
  - **단점:** ID가 존재하지 않으면 오류 발생.

In [33]:
# 업데이트할 새 Document 객체 생성
updated_document_1 = Document(
    page_content="인공지능(AI)은 컴퓨터 과학의 핵심 분야로, 기계학습과 딥러닝을 포괄합니다.", # 내용 수정
    metadata={"source": "AI_textbook_v2", "chapter": "Chapter 1"}, # 메타데이터 수정
)

updated_document_2 = Document(
    page_content="머신러닝(ML)은 데이터로부터 학습하여 예측과 결정을 내리는 인공지능의 중요한 하위 분야입니다.",
    metadata={"source": "AI_textbook_v2", "chapter": "Chapter 2"},
)

updated_document_3 = Document(
    page_content="딥러닝(DL)은 머신러닝의 한 갈래로, 복잡한 패턴 인식을 위해 심층 신경망을 사용합니다.",
    metadata={"source": "AI_textbook_v2", "chapter": "Chapter 3"},
)

# 단일 문서 업데이트 (ID: DOC_1)
chroma_db.update_document(document_id="DOC_1", document=updated_document_1)
print("DOC_1 업데이트 완료.")

# 여러 문서 한 번에 업데이트 (ID: DOC_2, DOC_3)
chroma_db.update_documents(
    ids=["DOC_2", "DOC_3"],
    documents=[updated_document_2, updated_document_3]
)
print("DOC_2, DOC_3 업데이트 완료.")

DOC_1 업데이트 완료.
DOC_2, DOC_3 업데이트 완료.


In [34]:
# 업데이트 후 문서 검색 테스트
query = "인공지능과 머신러닝의 관계는?"
results = chroma_db.similarity_search(query, k=2)

print(f"\n쿼리: {query}")
print("업데이트 후 검색된 유사 문서:")
for doc in results:
    print(f"- {doc.page_content} [출처: {doc.metadata['source']}, {doc.metadata['chapter']}]")


쿼리: 인공지능과 머신러닝의 관계는?
업데이트 후 검색된 유사 문서:
- 머신러닝(ML)은 데이터로부터 학습하여 예측과 결정을 내리는 인공지능의 중요한 하위 분야입니다. [출처: AI_textbook_v2, Chapter 2]
- 인공지능(AI)은 컴퓨터 과학의 핵심 분야로, 기계학습과 딥러닝을 포괄합니다. [출처: AI_textbook_v2, Chapter 1]


- 문서 삭제: `vector_store.delete(ids)`
  - 제공된 ID 리스트에 해당하는 문서를 삭제함.
  - **장점:** 특정 문서들을 선택적으로 제거 가능.
  - **단점:** ID가 존재하지 않아도 오류 없이 넘어가므로, 삭제 확인 필요.

In [35]:
# DOC_5 문서 삭제
chroma_db.delete(ids=["DOC_5"])
print("DOC_5 삭제 시도 완료. (존재했다면 삭제됨)")

DOC_5 삭제 시도 완료. (존재했다면 삭제됨)


In [36]:
# 삭제 후 컬렉션 확인 (DOC_5가 사라졌는지 확인)
pprint(chroma_db.get(ids=['DOC_1', 'DOC_2', 'DOC_3', 'DOC_4', 'DOC_5'])) # 모든 ID로 확인

{'data': None,
 'documents': ['인공지능(AI)은 컴퓨터 과학의 핵심 분야로, 기계학습과 딥러닝을 포괄합니다.',
               '머신러닝(ML)은 데이터로부터 학습하여 예측과 결정을 내리는 인공지능의 중요한 하위 분야입니다.',
               '딥러닝(DL)은 머신러닝의 한 갈래로, 복잡한 패턴 인식을 위해 심층 신경망을 사용합니다.',
               '자연어 처리는 컴퓨터가 인간의 언어를 이해하고 생성하는 기술입니다.'],
 'embeddings': None,
 'ids': ['DOC_1', 'DOC_2', 'DOC_3', 'DOC_4'],
 'included': [<IncludeEnum.documents: 'documents'>,
              <IncludeEnum.metadatas: 'metadatas'>],
 'metadatas': [{'chapter': 'Chapter 1', 'source': 'AI_textbook_v2'},
               {'chapter': 'Chapter 2', 'source': 'AI_textbook_v2'},
               {'chapter': 'Chapter 3', 'source': 'AI_textbook_v2'},
               {'chapter': 'Chapter 4', 'source': 'AI_textbook'}],
 'uris': None}


`(3) 문서 검색`

Chroma DB에서 문서를 검색하는 다양한 방법을 살펴봄.

- 유사도 검색: `similarity_search(query, k, filter)`
  - 주어진 쿼리 문자열과 가장 유사한 `k`개의 문서를 반환함.
  - `filter` 옵션을 사용하여 메타데이터 기준으로 검색 대상을 한정할 수 있음.
  - **장점:** 가장 기본적인 검색 방식, 메타데이터 필터링으로 검색 범위 조절 가능.
  - **단점:** 단순 유사도만 고려, 검색 결과의 다양성이 부족할 수 있음.

In [37]:
query = "인공지능과 머신러닝의 차이점은 무엇인가요?"
results = chroma_db.similarity_search(
    query,
    k=2, # 상위 2개 결과
    filter={"source": "AI_textbook_v2"} # 'source'가 'AI_textbook_v2'인 문서만 대상
)

print("유사도 검색 결과 (필터 적용):")
for doc in results:
    print(f"- {doc.page_content} [출처: {doc.metadata['source']}, {doc.metadata['chapter']}]")

유사도 검색 결과 (필터 적용):
- 머신러닝(ML)은 데이터로부터 학습하여 예측과 결정을 내리는 인공지능의 중요한 하위 분야입니다. [출처: AI_textbook_v2, Chapter 2]
- 인공지능(AI)은 컴퓨터 과학의 핵심 분야로, 기계학습과 딥러닝을 포괄합니다. [출처: AI_textbook_v2, Chapter 1]


- 유사도 점수가 포함된 검색: `similarity_search_with_score(query, k, filter)`
  - 검색된 문서와 함께 유사도 점수(distance score)를 반환함.
  - Chroma의 기본 거리 메트릭(L2, 코사인 등)에 따라 점수가 계산되며, 일반적으로 **점수가 낮을수록 더 유사함** (거리가 가까움을 의미).
  - **장점:** 유사도 수준을 정량적으로 파악 가능, 임계값 설정 등에 활용.
  - **단점:** 점수의 스케일이 거리 메트릭에 따라 달라 해석에 주의 필요.

In [38]:
query = "딥러닝은 어떤 분야에서 사용되나요?"
results_with_score = chroma_db.similarity_search_with_score(
    query,
    k=2,
    filter={"source": "AI_textbook_v2"}
)

print("점수가 포함된 유사도 검색 결과:\n")
for doc, score in results_with_score:
    print(f"- 점수 (거리): {score:.4f} (낮을수록 유사)") # 점수가 낮을수록 유사
    print(f"  내용: {doc.page_content}")
    print(f"  [출처: {doc.metadata['source']}, {doc.metadata['chapter']}]")
    print()

점수가 포함된 유사도 검색 결과:

- 점수 (거리): 0.6154 (낮을수록 유사)
  내용: 딥러닝(DL)은 머신러닝의 한 갈래로, 복잡한 패턴 인식을 위해 심층 신경망을 사용합니다.
  [출처: AI_textbook_v2, Chapter 3]

- 점수 (거리): 0.7401 (낮을수록 유사)
  내용: 인공지능(AI)은 컴퓨터 과학의 핵심 분야로, 기계학습과 딥러닝을 포괄합니다.
  [출처: AI_textbook_v2, Chapter 1]



- 관련성 점수가 포함된 검색: `similarity_search_with_relevance_scores(query, k, filter)`
  - 문서와 함께 0에서 1 사이의 정규화된 관련성 점수(relevance score)를 반환함.
  - **점수가 높을수록 더 관련성이 높음** (1에 가까울수록 관련성이 높음).
  - 내부적으로 거리 점수를 유사도 점수로 변환하는 로직을 포함할 수 있음.
  - **장점:** 직관적인 점수 (0~1 스케일), 임계값 설정 용이.
  - **단점:** 변환 방식이 내부적으로 결정되어 세밀한 제어는 어려울 수 있음.

In [39]:
query = "딥러닝은 어떤 분야에서 사용되나요?"
results_with_relevance = chroma_db.similarity_search_with_relevance_scores(
    query,
    k=2,
    filter={"source": "AI_textbook_v2"}
)

print(f"쿼리: {query}")
print("\n검색 결과 (관련성 점수 포함):")
for doc, score in results_with_relevance:
    print(f"- 관련성 점수: {score:.4f} (높을수록 관련성 높음)") # 점수가 높을수록 관련성 높음
    print(f"  내용: {doc.page_content}")
    print(f"  [출처: {doc.metadata['source']}, {doc.metadata['chapter']}]")
    print()

쿼리: 딥러닝은 어떤 분야에서 사용되나요?

검색 결과 (관련성 점수 포함):
- 관련성 점수: 0.5648 (높을수록 관련성 높음)
  내용: 딥러닝(DL)은 머신러닝의 한 갈래로, 복잡한 패턴 인식을 위해 심층 신경망을 사용합니다.
  [출처: AI_textbook_v2, Chapter 3]

- 관련성 점수: 0.4766 (높을수록 관련성 높음)
  내용: 인공지능(AI)은 컴퓨터 과학의 핵심 분야로, 기계학습과 딥러닝을 포괄합니다.
  [출처: AI_textbook_v2, Chapter 1]



`(4) 벡터 저장소 로드`

- `persist_directory`에 저장된 Chroma DB를 다시 로드하여 사용할 수 있음.
- 이전에 사용한 `collection_name`, `embedding_function`, `persist_directory`를 동일하게 지정해야 함.
- **장점:** 애플리케이션 재시작 시 DB를 다시 구축할 필요 없음, 데이터 영속성 확보.
- **단점:** 저장 공간 차지, 로드 시 약간의 시간 소요 가능.

In [40]:
# 이전에 저장한 Chroma DB 로드
chroma_db2 = Chroma(
    collection_name="ai_smaple_collection", # 동일한 컬렉션 이름
    embedding_function=embeddings_model,    # 동일한 임베딩 함수
    persist_directory="./chroma_db",        # 동일한 저장 경로
)
print("저장된 Chroma DB 로드 완료.")

저장된 Chroma DB 로드 완료.


In [41]:
# 로드된 DB에서 검색 테스트
query = "딥러닝은 어떤 분야에서 사용되나요?"
results_loaded = chroma_db2.similarity_search_with_relevance_scores(
    query,
    k=2,
    filter={"source": "AI_textbook_v2"}
)

print(f"쿼리: {query}")
print("\n로드된 DB에서의 검색 결과 (관련성 점수 포함):")
for doc, score in results_loaded:
    print(f"- 관련성 점수: {score:.4f}")
    print(f"  내용: {doc.page_content}")
    print(f"  [출처: {doc.metadata['source']}, {doc.metadata['chapter']}]")
    print()

쿼리: 딥러닝은 어떤 분야에서 사용되나요?

로드된 DB에서의 검색 결과 (관련성 점수 포함):
- 관련성 점수: 0.5648
  내용: 딥러닝(DL)은 머신러닝의 한 갈래로, 복잡한 패턴 인식을 위해 심층 신경망을 사용합니다.
  [출처: AI_textbook_v2, Chapter 3]

- 관련성 점수: 0.4766
  내용: 인공지능(AI)은 컴퓨터 과학의 핵심 분야로, 기계학습과 딥러닝을 포괄합니다.
  [출처: AI_textbook_v2, Chapter 1]



### 1.2 FAISS (Facebook AI Similarity Search)

- Facebook (Meta)에서 개발한, 효율적인 벡터 유사도 검색 및 클러스터링을 위한 라이브러리임.
- 특히 대용량 벡터 데이터셋에서 빠른 검색 속도를 자랑함.
- CPU 버전은 `faiss-cpu`, GPU 버전은 `faiss-gpu` 패키지 설치 필요
- **장점:**
  - 검색 속도: 매우 빠름. 대용량 데이터셋에 적합.
  - 다양한 인덱싱 옵션: 데이터 특성 및 요구사항(속도 vs 정확도)에 맞는 다양한 인덱스 사용 가능 (예: IndexFlatL2, IndexIVFPQ).
  - 메모리 효율성: 일부 인덱스는 벡터 압축을 통해 메모리 사용량 절감 가능.
- **단점:**
  - 사용성: Chroma보다 초기 설정 및 관리가 다소 복잡할 수 있음.
  - 문서 업데이트/삭제: 직접적인 업데이트 기능이 제한적. 일반적으로 삭제 후 재추가 또는 인덱스 재구축 방식을 사용.
  - LangChain 통합: `InMemoryDocstore`와 `index_to_docstore_id` 매핑 관리가 필요할 수 있음.

`(1) 벡터 저장소 초기화`

- FAISS 인덱스를 먼저 생성하고, 이를 LangChain의 `FAISS` 클래스와 결합함.
- `faiss.IndexFlatL2`: 가장 기본적인 L2 거리(유클리드 거리) 기반의 정확한 검색 인덱스. 모든 벡터와 직접 비교하므로 데이터가 많아지면 느릴 수 있음.
- 임베딩 차원 수(`embeddings_model.embed_query("hello world")`의 길이)를 알아야 인덱스 초기화 가능.

In [42]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
embeddings_model = HuggingFaceEmbeddings(model_name="BAAI/bge-m3") # Chroma에서 사용한 모델 재활용

import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore # 문서를 메모리에 저장
from langchain_community.vectorstores import FAISS

# 임베딩된 쿼리 벡터의 차원 수 확인 (FAISS 인덱스 생성 시 필요)
embedding_dim = len(embeddings_model.embed_query("임베딩 차원 확인용 텍스트"))
print(f"사용할 임베딩 모델의 차원 수: {embedding_dim}")

# FAISS 인덱스 초기화 (L2 거리 사용)
faiss_index = faiss.IndexFlatL2(embedding_dim) # embedding_dim: 벡터의 차원 수
print(f"FAISS 인덱스 ({type(faiss_index).__name__}) 초기화 완료.")

사용할 임베딩 모델의 차원 수: 1024
FAISS 인덱스 (IndexFlatL2) 초기화 완료.


In [43]:
# FAISS 벡터 저장소의 벡터 차원 수 확인
print(f"FAISS 인덱스의 차원 수 (d): {faiss_index.d}")
# FAISS 벡터 저장소에 저장된 벡터의 총 개수 (초기에는 0)
print(f"FAISS 인덱스 내 벡터 수 (ntotal): {faiss_index.ntotal}")

FAISS 인덱스의 차원 수 (d): 1024
FAISS 인덱스 내 벡터 수 (ntotal): 0


In [44]:
# LangChain FAISS 래퍼 생성
faiss_db = FAISS(
    embedding_function=embeddings_model, # 임베딩 함수
    index=faiss_index,                   # 미리 생성한 FAISS 인덱스
    docstore=InMemoryDocstore(),         # 문서 내용을 저장할 Docstore (메모리 기반)
    index_to_docstore_id={},           # FAISS 인덱스 ID와 Docstore ID 매핑 (초기에는 비어있음)
)
print("LangChain FAISS 생성 완료.")

# 저장된 문서의 갯수 확인 (아직 문서 추가 전)
print(f"FAISS DB 내 문서 수 (index.ntotal): {faiss_db.index.ntotal}")

LangChain FAISS 생성 완료.
FAISS DB 내 문서 수 (index.ntotal): 0


`(2) 벡터 저장소 관리`

- 문서 추가: `vector_store.add_documents(documents, ids)`
  - Chroma와 유사하게 `Document` 객체와 ID를 사용하여 문서를 추가함.
  - 내부적으로 `docstore`에 문서 내용을 저장하고, `index`에 벡터를 추가하며, `index_to_docstore_id`에 매핑 정보를 기록함.
  - **장점:** 여러 문서 일괄 추가 가능.
  - **단점:** FAISS는 직접적인 문서 업데이트 기능이 약하므로, 변경 시 삭제 후 재추가가 일반적임.

In [45]:
from langchain_core.documents import Document

# Chroma에서 사용한 샘플 문서 내용 재활용
documents_content = [
    "인공지능은 컴퓨터 과학의 한 분야입니다.",
    "머신러닝은 인공지능의 하위 분야입니다.",
    "딥러닝은 머신러닝의 한 종류입니다.",
    "자연어 처리는 컴퓨터가 인간의 언어를 이해하고 생성하는 기술입니다.",
    "컴퓨터 비전은 컴퓨터가 디지털 이미지나 비디오를 이해하는 방법을 연구합니다."
]

# Document 객체 생성
doc_objects = []
for i, content in enumerate(documents_content, start=1):
    doc = Document(
        page_content=content,
        metadata={"source": "AI_textbook_FAISS", "chapter": f"Chapter {i}"}, # 메타데이터
    )
    doc_objects.append(doc)

# 고유 ID 리스트 생성
doc_ids = [f"FAISS_DOC_{i}" for i in range(1, len(doc_objects) + 1)]

# 문서를 FAISS 벡터 저장소에 추가
added_doc_ids = faiss_db.add_documents(documents=doc_objects, ids=doc_ids)

print(f"{len(added_doc_ids)}개의 문서가 성공적으로 FAISS에 추가되었습니다.")
print("추가된 문서 ID:", added_doc_ids)

5개의 문서가 성공적으로 FAISS에 추가되었습니다.
추가된 문서 ID: ['FAISS_DOC_1', 'FAISS_DOC_2', 'FAISS_DOC_3', 'FAISS_DOC_4', 'FAISS_DOC_5']


In [46]:
# 추가 후 저장된 문서의 갯수 확인
print(f"FAISS DB 내 문서 수 (index.ntotal): {faiss_db.index.ntotal}")
# Docstore 내용 확인 (일부만)
print("Docstore 일부 내용:", list(faiss_db.docstore._dict.items())[:2])
# Index to Docstore ID 매핑 확인 (일부만)
print("Index to Docstore ID 매핑 일부:", list(faiss_db.index_to_docstore_id.items())[:5])

FAISS DB 내 문서 수 (index.ntotal): 5
Docstore 일부 내용: [('FAISS_DOC_1', Document(id='FAISS_DOC_1', metadata={'source': 'AI_textbook_FAISS', 'chapter': 'Chapter 1'}, page_content='인공지능은 컴퓨터 과학의 한 분야입니다.')), ('FAISS_DOC_2', Document(id='FAISS_DOC_2', metadata={'source': 'AI_textbook_FAISS', 'chapter': 'Chapter 2'}, page_content='머신러닝은 인공지능의 하위 분야입니다.'))]
Index to Docstore ID 매핑 일부: [(0, 'FAISS_DOC_1'), (1, 'FAISS_DOC_2'), (2, 'FAISS_DOC_3'), (3, 'FAISS_DOC_4'), (4, 'FAISS_DOC_5')]


- 문서 삭제: `vector_store.delete(ids)`
  - 제공된 ID 리스트에 해당하는 문서를 `docstore`에서 삭제하고, `index`에서도 해당 벡터를 (논리적으로) 삭제함.
  - `IndexFlatL2` 같은 일부 인덱스는 실제 물리적 삭제가 아닌, 재구성(reconstruction) 전까지는 마킹만 해둘 수 있음. FAISS의 `remove_ids` 동작에 따름.
  - **장점:** 특정 문서 제거 가능.
  - **단점:** 인덱스 종류에 따라 삭제 성능/방식이 다를 수 있음. 대량 삭제 후에는 인덱스 재구축이 효율적일 수 있음.

In [47]:
# FAISS_DOC_5 문서 삭제
deleted_success = faiss_db.delete(ids=["FAISS_DOC_5"])
if deleted_success:
    print("FAISS_DOC_5 삭제 성공 (또는 ID가 원래 없었음).")
else:
    print("FAISS_DOC_5 삭제 실패. 해당 ID가 존재하지 않거나 삭제 중 문제 발생.")
    # Langchain FAISS delete는 성공 여부를 boolean으로 반환하는데, 실제로는 내부적으로 예외처리 될 수 있음.
    # 여기서는 삭제 후 ntotal을 확인하는 것이 더 확실함.

FAISS_DOC_5 삭제 성공 (또는 ID가 원래 없었음).


In [53]:
# 삭제 후 컬렉션(인덱스 내 벡터 수) 확인
print(f"삭제 후 FAISS DB 내 문서 수 (index.ntotal): {faiss_db.index.ntotal}")
# FAISS_DOC_5가 docstore에서 사라졌는지 확인
print("FAISS_DOC_5 존재 여부 (docstore):", 'FAISS_DOC_5' in faiss_db.docstore._dict)

삭제 후 FAISS DB 내 문서 수 (index.ntotal): 4
FAISS_DOC_5 존재 여부 (docstore): False


`(3) 문서 검색`

FAISS DB에서 문서를 검색하는 방법도 Chroma와 유사한 인터페이스를 제공함.

- 유사도 검색: `similarity_search(query, k, filter)`
  - Chroma와 동일한 방식으로 사용. `filter`는 메타데이터 기반 필터링으로, FAISS 래퍼가 검색 후 필터링을 수행함.
  - **장점:** LangChain 표준 인터페이스 사용, 필터링 가능.
  - **단점:** 필터링이 FAISS 인덱스 수준에서 이루어지는 것이 아니라, 검색된 결과에 대해 후처리로 필터링될 수 있어 성능에 영향 줄 수 있음 (특히 매우 많은 결과 중 필터링 시).

In [54]:
query = "인공지능과 머신러닝의 차이점은 무엇인가요?"
results_faiss = faiss_db.similarity_search(
    query,
    k=2,
    filter={"source": "AI_textbook_FAISS"} # 메타데이터 필터
)

print("FAISS 유사도 검색 결과:")
for doc in results_faiss:
    print(f"- {doc.page_content} [출처: {doc.metadata['source']}, {doc.metadata['chapter']}]")

FAISS 유사도 검색 결과:
- 머신러닝은 인공지능의 하위 분야입니다. [출처: AI_textbook_FAISS, Chapter 2]
- 딥러닝은 머신러닝의 한 종류입니다. [출처: AI_textbook_FAISS, Chapter 3]


- 유사도 점수가 포함된 검색: `similarity_search_with_score(query, k, filter)`
  - 검색된 문서와 함께 거리 점수(L2 거리 등)를 반환. **점수가 낮을수록 더 유사함**.
  - **장점:** 유사도 수준 정량적 파악.
  - **단점:** 점수 해석에 주의 필요 (거리 기반).

In [55]:
query = "딥러닝은 어떤 분야에서 사용되나요?"
results_faiss_score = faiss_db.similarity_search_with_score(
    query,
    k=2,
    filter={"source": "AI_textbook_FAISS"}
)

print("FAISS 점수가 포함된 유사도 검색 결과:\n")
for doc, score in results_faiss_score:
    print(f"- 점수 (거리): {score:.4f} (낮을수록 유사)")
    print(f"  내용: {doc.page_content}")
    print(f"  [출처: {doc.metadata['source']}, {doc.metadata['chapter']}]")
    print()

FAISS 점수가 포함된 유사도 검색 결과:

- 점수 (거리): 0.6517 (낮을수록 유사)
  내용: 딥러닝은 머신러닝의 한 종류입니다.
  [출처: AI_textbook_FAISS, Chapter 3]

- 점수 (거리): 0.8442 (낮을수록 유사)
  내용: 머신러닝은 인공지능의 하위 분야입니다.
  [출처: AI_textbook_FAISS, Chapter 2]



- 관련성 점수가 포함된 검색: `similarity_search_with_relevance_scores(query, k, filter)`
  - 0~1 사이의 정규화된 관련성 점수를 반환. **점수가 높을수록 더 관련성이 높음**.
  - **장점:** 직관적인 점수 (0~1 스케일).
  - **단점:** 내부 변환 로직에 의존.

In [56]:
query = "딥러닝은 어떤 분야에서 사용되나요?"
results_faiss_relevance = faiss_db.similarity_search_with_relevance_scores(
    query,
    k=2,
    filter={"source": "AI_textbook_FAISS"}
)

print(f"쿼리: {query}")
print("\nFAISS 검색 결과 (관련성 점수 포함):")
for doc, score in results_faiss_relevance:
    print(f"- 관련성 점수: {score:.4f} (높을수록 관련성 높음)")
    print(f"  내용: {doc.page_content}")
    print(f"  [출처: {doc.metadata['source']}, {doc.metadata['chapter']}]")
    print()

쿼리: 딥러닝은 어떤 분야에서 사용되나요?

FAISS 검색 결과 (관련성 점수 포함):
- 관련성 점수: 0.5392 (높을수록 관련성 높음)
  내용: 딥러닝은 머신러닝의 한 종류입니다.
  [출처: AI_textbook_FAISS, Chapter 3]

- 관련성 점수: 0.4031 (높을수록 관련성 높음)
  내용: 머신러닝은 인공지능의 하위 분야입니다.
  [출처: AI_textbook_FAISS, Chapter 2]



`(4) 로컬에 저장 및 로드`

- FAISS 인덱스와 `docstore` 내용을 로컬 파일 시스템에 저장하고, 필요시 다시 로드할 수 있음.
- `faiss_db.save_local("폴더명")`으로 저장, `FAISS.load_local("폴더명", embeddings_model)`로 로드함.
- `allow_dangerous_deserialization=True`: Pickle 등을 사용하여 객체를 직렬화/역직렬화할 때 보안상 위험이 있을 수 있음을 인지하고 허용하는 옵션임. 신뢰할 수 있는 데이터 소스에만 사용해야 함.
- **장점:** 인덱스 재구축 없이 재사용 가능, 데이터 영속화.
- **단점:** `allow_dangerous_deserialization` 사용 시 보안 유의, 저장/로드 시 I/O 발생.

In [57]:
# FAISS DB 로컬에 저장
save_folder = "faiss_ai_smaple_index"
faiss_db.save_local(save_folder)
print(f"FAISS DB가 '{save_folder}' 폴더에 저장되었습니다.")

FAISS DB가 'faiss_ai_smaple_index' 폴더에 저장되었습니다.


In [58]:
# 로컬에 저장된 FAISS 벡터 저장소 불러오기
faiss_db2 = FAISS.load_local(
    save_folder, 
    embeddings_model, 
    allow_dangerous_deserialization=True # 보안 위험 인지 후 사용
)
print(f"'{save_folder}' 폴더에서 FAISS DB 로드 완료.")

'faiss_ai_smaple_index' 폴더에서 FAISS DB 로드 완료.


In [59]:
# 로드된 DB의 docstore 내용 확인 (일부만)
print("로드된 FAISS DB의 Docstore 일부 내용:", list(faiss_db2.docstore._dict.items())[:2])
print(f"로드된 FAISS DB 내 문서 수 (index.ntotal): {faiss_db2.index.ntotal}")

로드된 FAISS DB의 Docstore 일부 내용: [('FAISS_DOC_1', Document(id='FAISS_DOC_1', metadata={'source': 'AI_textbook_FAISS', 'chapter': 'Chapter 1'}, page_content='인공지능은 컴퓨터 과학의 한 분야입니다.')), ('FAISS_DOC_2', Document(id='FAISS_DOC_2', metadata={'source': 'AI_textbook_FAISS', 'chapter': 'Chapter 2'}, page_content='머신러닝은 인공지능의 하위 분야입니다.'))]
로드된 FAISS DB 내 문서 수 (index.ntotal): 4


In [60]:
# 로드된 FAISS DB에서 검색 테스트
query = "딥러닝은 어떤 분야에서 사용되나요?"
results_faiss_loaded = faiss_db2.similarity_search_with_relevance_scores(
    query,
    k=2,
    filter={"source": "AI_textbook_FAISS"}
)

print(f"쿼리: {query}")
print("\n로드된 FAISS DB에서의 검색 결과 (관련성 점수 포함):")
for doc, score in results_faiss_loaded:
    print(f"- 관련성 점수: {score:.4f}")
    print(f"  내용: {doc.page_content}")
    print(f"  [출처: {doc.metadata['source']}, {doc.metadata['chapter']}]")
    print()

쿼리: 딥러닝은 어떤 분야에서 사용되나요?

로드된 FAISS DB에서의 검색 결과 (관련성 점수 포함):
- 관련성 점수: 0.5392
  내용: 딥러닝은 머신러닝의 한 종류입니다.
  [출처: AI_textbook_FAISS, Chapter 3]

- 관련성 점수: 0.4031
  내용: 머신러닝은 인공지능의 하위 분야입니다.
  [출처: AI_textbook_FAISS, Chapter 2]

